### NEWS 2 Predictive Model Python Cleaning

By Matt Stammers, Megha Bhandari and Florina Borca - UHS Digital

This script prepares the data for the NEWS2 COVID Predictive Model in readiness for comparison with the outcomes from the King's model.

In [ ]:
# Import the Key Packages

import datetime
from datetime import timedelta
import os
import numpy as np
import pandas as pd
from joblib import load
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import (make_scorer,
                             confusion_matrix,
                             roc_auc_score,
                             recall_score)

In [ ]:
# Import the Directories

Raw_Dir = "Z:/Trust/Information team (SHARE)/COVID-19 datasets/National Early Warning Score (NEWS)/Datasets/Raw_Dir/"
Processed_Dir = "Z:/Trust/Information team (SHARE)/COVID-19 datasets/National Early Warning Score (NEWS)/Datasets/Processed_Dir/"
Output_Dir = "Z:/Trust/Information team (SHARE)/COVID-19 datasets/National Early Warning Score (NEWS)/Datasets/Output_Dir/"

fn_base = 'CHESS_ML_Database2305.csv'
fn_path='Pathology_20200518.xlsx'
fn_news='NEWS2_score_20200523.xlsx'
fn_alb='ALB.xlsx'
fn_labs='Labs.csv'
fn_adms='CamisWardAndBed.csv'
fn_virology='2020-05-22 COVID-19 Results.csv'
fn_resp = 'Respiratory_rate.xlsx'
fn_sats = 'Oxygen_saturation.xlsx'

In [ ]:
# Import the main data files
df_virology=pd.read_csv(Raw_Dir + fn_virology)
df_base=pd.read_csv(Processed_Dir + fn_base)
df_path=pd.read_excel(Raw_Dir + fn_path, parse_dates = True)
df_news=pd.read_excel(Raw_Dir + fn_news)
df_alb=pd.read_excel(Raw_Dir + fn_alb)
df_labs=pd.read_csv(Raw_Dir+ fn_labs, sep='|') # You can also use delimiter = '|' they do the same thing
df_adms=pd.read_csv(Raw_Dir+ fn_adms, sep='|')
df_resp=pd.read_excel(Raw_Dir + fn_resp)
df_sats=pd.read_excel(Raw_Dir + fn_sats)

In [ ]:
# Adjust settings to see entire frame:

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None

In [ ]:
df_key = df_base[["HospitalNumber", "DateOfHospitalAdmission", "Date & Time of Receipt"]]

In [ ]:
df_key['DateOfHospitalAdmission'] = pd.to_datetime(df_key['DateOfHospitalAdmission'], dayfirst = True)
df_key['Date & Time of Receipt'] = pd.to_datetime(df_key['Date & Time of Receipt'], dayfirst = True)

In [ ]:
df_key['Adm_Test_Timelag'] = df_key['Date & Time of Receipt'] - df_key.DateOfHospitalAdmission

In [ ]:
from datetime import timedelta
df_key['DateOfHospitalAdmission+5']= df_key['DateOfHospitalAdmission'] + timedelta(days=3)
df_key['DateOfHospitalAdmission-2']= df_key['DateOfHospitalAdmission'] + timedelta(days=0)
df_key['Date & Time of Receipt+5'] = df_key['Date & Time of Receipt']  + timedelta(days=3)
df_key['Date & Time of Receipt-2'] = df_key['Date & Time of Receipt']  + timedelta(days=0)

In [ ]:
df_key.head()

In [ ]:
# Build a massive dataframe with all the key results in it

#df_path2
#df_news2
#df_alb2
#df_labs2
#df_resp
#df_sats

In [ ]:
df_path2 = df_path[['PATIENT_NO', 'PATHOLOGY_SPECIMEN_DATE', 'PATHOLOGY_TEST_CODE', 'PATHOLOGY_RESULT_NUMERIC']]
df_news2 = df_news[['HospitalNumber', 'Time', 'parameterName', 'value']]
df_alb2 = df_alb[['PATIENT_NO', 'PATHOLOGY_SPECIMEN_DATE', 'PATHOLOGY_TEST_CODE', 'PATHOLOGY_RESULT_NUMERIC']]
df_labs2 = df_labs[['PATIENT_NUMBER', 'SPECIMEN_DATE', 'TEST_CODE', 'RESULT_VALUE']]
df_sats2 = df_sats[['HospitalNumber', 'ValidationTime', 'ParameterName', 'Value']]
df_resp2 = df_resp[['HospitalNumber', 'ValidationTime', 'ParameterName', 'Value']]

In [ ]:
df_path2.columns = ['PATIENT_NO', 'DATE', 'TEST_CODE', 'RESULT']
df_news2.columns = ['PATIENT_NO', 'DATE', 'TEST_CODE', 'RESULT']
df_alb2.columns = ['PATIENT_NO', 'DATE', 'TEST_CODE', 'RESULT']
df_labs2.columns = ['PATIENT_NO', 'DATE', 'TEST_CODE', 'RESULT']
df_sats2.columns = ['PATIENT_NO', 'DATE', 'TEST_CODE', 'RESULT']
df_resp2.columns = ['PATIENT_NO', 'DATE', 'TEST_CODE', 'RESULT']

In [ ]:
# Now join them all

df_joined = pd.concat([df_path2, df_news2])
df_joined2 = pd.concat([df_joined, df_alb2])
df_joined3 = pd.concat([df_joined2, df_labs2])
df_joined4 = pd.concat([df_joined3, df_sats2])
df_joined5 = pd.concat([df_joined4, df_resp2])

In [ ]:
df_joined5.DATE = pd.to_datetime(df_joined5.DATE,dayfirst=True)

In [ ]:
df_joined5.info()

In [ ]:
# Function to Repair Hospital Numbers

def fullno(x):
    if len(x)==7:
        return x
    elif len(x)==6:
        return ''.join('0'+x)
    elif len(x)==5:
        return ''.join('00'+x)
    elif len(x)==4:
        return ''.join('000'+x)
    else:
        return ''

In [ ]:
# Then repair hospital numbers

df_joined5.PATIENT_NO = df_joined5.PATIENT_NO.astype(str)
df_joined5.PATIENT_NO = df_joined5.PATIENT_NO.apply(fullno)

In [ ]:
df_joined5.shape

In [ ]:
# Then join them together

df_full = pd.merge(df_key, df_joined5, left_on='HospitalNumber', right_on ='PATIENT_NO', how ='left')

In [ ]:
df_full.info()

In [ ]:
df_full.TEST_CODE.unique()

In [ ]:
key_tests = ['*Respiration rate', 'Peripheral oxygen saturation' ,'Total Obs Score', 'NEUT', 'ALB2', 'AST', 'FER2', 'TRIG',
       'CRP', 'CR2', 'GFR3', 'HST1', 'Hb', 'LYM', 'PLT', 'WBC', 'DD2',
       'FIBD', '25VD', 'AKIA']

In [ ]:
df_full2 = df_full[df_full['TEST_CODE'].isin(key_tests)]

In [ ]:
# To Clean some of the blood tests you may need the following:

def Whitespace_Strip(x):
    return x.replace(' ','')

def Dot_Remover(x):
    if x == '.':
        return ''
    elif x == '..':
        return ''
    else:
        return x

In [ ]:
df_full2['RESULT'].unique()

In [ ]:
# Clean the bloods to convert to Float:

df_full2['RESULT'] = df_full2['RESULT'].astype(str)
df_full2['RESULT'] = df_full2['RESULT'].apply(lambda x: ''.join(i for i in x if i.isdigit() or i == '.'))
df_full2['RESULT'] = df_full2['RESULT'].apply(Dot_Remover)
df_full2['RESULT'] = df_full2['RESULT'].apply(Whitespace_Strip)
df_full2['RESULT'] = df_full2['RESULT'].replace('.', '')
df_full2['RESULT'] = df_full2['RESULT'].replace('', 'NaN')
df_full2.dropna(subset = ["RESULT"], inplace=True)
df_full2['RESULT'] = df_full2['RESULT'].astype(float)

In [ ]:
# There are some irritating nans hiding thus why I had to find them amongst the thousands of rows!
pd.set_option('display.max_rows', None)

In [ ]:
#df_full3['RESULT'].explode().value_counts()

In [ ]:
df_full2.info()

In [ ]:
df_full2['COVID_TEST_TIMELAG'] = df_full2['DATE'] - df_full2['Date & Time of Receipt']

In [ ]:
df_adm_filtered = df_full2[df_full2['DATE'].between(df_full2['DateOfHospitalAdmission-2'], df_full2['DateOfHospitalAdmission+5'])]
df_lab_filtered = df_full2[df_full2['DATE'].between(df_full2['Date & Time of Receipt-2'], df_full2['Date & Time of Receipt+5'])]

In [ ]:
df_adm_filtered.shape

In [ ]:
df_lab_filtered.shape

In [ ]:
df_lab_filtered.head()

In [ ]:
df_lab_filtered.columns

In [ ]:
df_lab_filtered2 = df_lab_filtered[['HospitalNumber','TEST_CODE', 'RESULT']]

In [ ]:
df_lab_filtered_first_grouped = df_lab_filtered2.groupby(['HospitalNumber','TEST_CODE']).agg('max').transpose().stack(0).reset_index()

In [ ]:
df_lab_filtered_first_grouped.columns

In [ ]:
df_lab_filtered_first_grouped = df_lab_filtered_first_grouped[['HospitalNumber', '*Respiration rate', '25VD', 'AKIA', 'ALB2', 'AST', 'CR2', 'CRP', 'DD2', 'FER2', 'FIBD', 'GFR3', 'HST1', 'Hb', 'LYM', 'NEUT', 'PLT', 'Peripheral oxygen saturation', 'TRIG', 'Total Obs Score', 'WBC']]

In [ ]:
# This is what we needed
df_lab_filtered_first_grouped.head()

In [ ]:
df_lab_filtered_counts_grouped = df_lab_filtered2.groupby(['HospitalNumber','TEST_CODE']).agg('count').transpose().stack(0).reset_index()

In [ ]:
df_lab_filtered_counts_grouped = df_lab_filtered_counts_grouped[['HospitalNumber', '*Respiration rate', '25VD', 'AKIA', 'ALB2', 'AST', 'CR2', 'CRP', 'DD2', 'FER2', 'FIBD', 'GFR3', 'HST1', 'Hb', 'LYM', 'NEUT', 'PLT', 'Peripheral oxygen saturation', 'TRIG', 'Total Obs Score', 'WBC']]

In [ ]:
df_lab_filtered_counts_grouped.apply(lambda x: x.isnull().value_counts())

In [ ]:
# Ok this one is ready
df_lab_filtered_counts_grouped.shape

In [ ]:
df_adm_filtered2 = df_adm_filtered[['HospitalNumber','TEST_CODE', 'RESULT']]

In [ ]:
df_adm_filtered_first_grouped = df_adm_filtered2.groupby(['HospitalNumber','TEST_CODE']).agg('first').transpose().stack(0).reset_index()

In [ ]:
df_adm_filtered_first_grouped.columns

In [ ]:
df_adm_filtered_first_grouped = df_adm_filtered_first_grouped[['HospitalNumber', '*Respiration rate', '25VD', 'AKIA', 'ALB2', 'AST', 'CR2', 'CRP', 'DD2', 'FER2', 'FIBD', 'GFR3', 'HST1', 'Hb', 'LYM', 'NEUT', 'PLT', 'Peripheral oxygen saturation', 'TRIG', 'Total Obs Score', 'WBC']]

In [ ]:
# This is what we needed
df_adm_filtered_first_grouped.head()

In [ ]:
df_adm_filtered_counts_grouped = df_adm_filtered2.groupby(['HospitalNumber','TEST_CODE']).agg('count').transpose().stack(0).reset_index()

In [ ]:
df_adm_filtered_counts_grouped = df_adm_filtered_counts_grouped[['HospitalNumber', '*Respiration rate', '25VD', 'AKIA', 'ALB2', 'AST', 'CR2', 'CRP', 'DD2', 'FER2', 'FIBD', 'GFR3', 'HST1', 'Hb', 'LYM', 'NEUT', 'PLT', 'Peripheral oxygen saturation', 'TRIG', 'Total Obs Score', 'WBC']]

In [ ]:
df_adm_filtered_counts_grouped.apply(lambda x: x.isnull().value_counts())

In [ ]:
# Ok this one is ready
df_adm_filtered_counts_grouped.shape

In [ ]:
df_lab_filtered_first_grouped.info()

In [ ]:
df_final = pd.merge(df_base, df_lab_filtered_first_grouped, on='HospitalNumber', how='left')

In [ ]:
df_final.shape

In [ ]:
df_final.columns.values

In [ ]:
df_final = df_final[['NHSNumber', 'HospitalNumber', 'Forename', 'Surname', 'DOB',
       'Ethnicity', 'Postcode', 'Sex', 'DateOfOnset', 'SwabDate',
       'LabTestDate', 'SpecimenType', 'OtherSpecimenType',
       'LabResultCOVID19', 'LabResultInfluenzaA_H1N1pdm2009',
       'LabResultInfluenzaA_H3N2', 'LabResultInfluenzaB',
       'LabResultInfluenzaA_Non_Subtyped',
       'LabResultInfluenzaA_Unsubtypable', 'LabResultRSV',
       'LabResultOther', 'LabResultOtherDetails', 'AdmissionRelatedToFlu',
       'AdmissionRelatedToRSV', 'AdmissionRelatedToCOVID19',
       'AdmittedFrom', 'AdmittedFromDetails', 'DateOfHospitalAdmission',
       'HourOfHospitalAdmission', 'MinuteOfHospitalAdmission',
       'AdmittedToICU_HDU', 'DateOfICU_HDUAdmission',
       'HourOfAdmissionToHDU_ICU', 'MinuteOfAdmissionToHDU_ICU',
       'DateOfLeavingICU_HDU', 'ComplicationsViralPneumonia',
       'ComplicationsSecondaryBacterialPneumonia', 'ComplicationsARDS',
       'ComplicationsUnknown', 'ComplicationsOtherCoInfections',
       'ComplicationsOther', 'IfComplicationsOtherSpecifyDetails',
       'IfOtherCoInfectionsSpecifyOrganism',
       'IfOtherCoInfectionsSpecifyTestDate', 'IfSBPSpecifyOrganismType',
       'IfSBPOrganismTypeOtherSpecify', 'SBPDate',
       'RespiratorySupportNone',
       'RespiratorySupportOxygenViaCannulaeOrMask',
       'RespiratorySupportHighFlowNasalOxygen',
       'RespiratorySupportNonInvasiveMechanicalVentilation',
       'RespiratorySupportInvasiveMechanicalVentilation',
       'RespiratorySupportECMO',
       'IfMechanicalInvasiveVentilationSpecifyDurationInDays',
       'SpecificCOVID19Treatment', 'ChronicRespiratoryDisease',
       'IfChronicRespiratoryDiseaseSpecifyCondition', 'Asthma',
       'IfAsthmaSpecifyCondition', 'ChronicHeartDisease',
       'IfChronicHeartDiseaseSpecifyCondition', 'Hypertension',
       'IfHypertensionSpecifyCondition', 'ChronicRenalDisease',
       'IfChronicRenalDiseaseSpecifyCondition', 'ChronicLiverDisease',
       'IfChronicLiverDiseaseSpecifyCondition',
       'ChronicNeurologicalDisease',
       'IfChronicNeurologicalDiseaseSpecifyCondition', 'Diabetes',
       'IfDiabetesSpecifyType', 'ImmunosuppressionTreatmentRelated',
       'IfImmunosuppressionTreatmentRelatedSpecifyCondition',
       'ImmunosuppressionDiseaseRelated',
       'IfImmunosuppressionDiseaseRelatedSpecifyCondition',
       'ObesityClinicallyApparent', 'ObesityBMI', 'Pregnant',
       'IfPregnantSpecifyGestationWeek',
       'TravelIn14DaysBeforeDiseaseOnset',
       'IfTravelIn14DaysBeforeDiseaseOnsetSpecifyTravelDestinationAndReturnDate',
       'PrematurityLessThan37Weeks', 'WorksAsAHealthcareWorker',
       'ContactWithConfirmedCase14DaysBeforeOnset',
       'UnderLyingConditionsOther',
       'IfUnderLyingConditionsOtherSpecifyCondition', 'FinalOutcome',
       'DateOfFinalOutcome', 'TransferDestination',
       'TransferLocationDetails', 'CauseOfDeath',
       'IfOtherCauseOfDeathSpecify', 'GeneralComments',
       'Date & Time of Receipt', '*Respiration rate', '25VD', 'AKIA',
       'ALB2', 'AST', 'CR2', 'CRP', 'DD2', 'FER2', 'FIBD', 'GFR3', 'HST1',
       'Hb', 'LYM', 'NEUT', 'PLT', 'Peripheral oxygen saturation', 'TRIG',
       'Total Obs Score', 'WBC']]

In [ ]:
df_final['DOB'] = pd.to_datetime(df_final['DOB'], dayfirst = True)
df_final['Date & Time of Receipt'] = pd.to_datetime(df_final['Date & Time of Receipt'], dayfirst = True)
df_final['DateOfHospitalAdmission'] = pd.to_datetime(df_final['DateOfHospitalAdmission'], dayfirst = True)
df_final['DateOfICU_HDUAdmission'] = pd.to_datetime(df_final['DateOfICU_HDUAdmission'], dayfirst = True)
df_final['DateOfLeavingICU_HDU'] = pd.to_datetime(df_final['DateOfLeavingICU_HDU'], dayfirst = True)
df_final['DateOfFinalOutcome'] = pd.to_datetime(df_final['DateOfFinalOutcome'], dayfirst = True)

In [ ]:
df_final.info()

In [ ]:
df_final.apply(lambda x: x.isnull().value_counts())

In [ ]:
#df_final['Final_Outcome_Time'] = df_final['Date & Time of Receipt'] + timedelta(days=14)

df_final['Final_Outcome_Time'] = df_final['Date & Time of Receipt'] + timedelta(days=10)

In [ ]:
df_final['Still_Admitted'] = df_final['Final_Outcome_Time'].between(df_final['DateOfHospitalAdmission'], df_final['DateOfFinalOutcome'])

In [ ]:
df_final['In_ICU'] = df_final['Final_Outcome_Time'].between(df_final['DateOfICU_HDUAdmission'], df_final['DateOfLeavingICU_HDU'])

In [ ]:
df_final['index_date']= df_final['Date & Time of Receipt']- timedelta(days = 4)

In [ ]:
df_final['Outcomed'] = df_final['DateOfFinalOutcome'].between(df_final['index_date'], df_final['Final_Outcome_Time'])

In [ ]:
df_final['Outcomed'] = df_final['Outcomed'].astype(str)

In [ ]:
df_final['Outcome'] = df_final['FinalOutcome'] + df_final['Outcomed'] 

In [ ]:
df_final['Outcome'] = df_final['Outcome'].apply(lambda x: x.replace('True', ''))

In [ ]:
df_final['Outcome'].unique()

In [ ]:
df_final['Outcome'] = df_final['Outcome'].apply(lambda x: x.replace('DeathFalse', ''))
df_final['Outcome'] = df_final['Outcome'].apply(lambda x: x.replace('DischargedFalse', ''))

In [ ]:
df_final['Outcome'].unique()

In [ ]:
df_final['In_ICU'] = df_final['In_ICU'].astype(str)
df_final['Still_Admitted'] = df_final['Still_Admitted'].astype(str)

In [ ]:
df_final['Still_Admitted'] = df_final['Still_Admitted'].apply(lambda x: x.replace('False', ''))

In [ ]:
df_final['Outcome'] = df_final['Outcome'] + df_final['Still_Admitted']

In [ ]:
df_final['Outcome'].unique()

In [ ]:
df_final['Outcome'] = df_final['Outcome'].apply(lambda x: x.replace('True','Inpatient'))

In [ ]:
df_final['Outcome'].unique()

In [ ]:
df_final['In_ICU']  = df_final['In_ICU'].apply(lambda x: x.replace('False', ''))

In [ ]:
df_final['In_ICU']  = df_final['In_ICU'].apply(lambda x: x.replace('True', 'In ICU'))

In [ ]:
df_final['Outcome'] = df_final['Outcome'] + df_final['In_ICU']

In [ ]:
df_final['Outcome'] = df_final['Outcome'].apply(lambda x: x.replace('InpatientIn ICU', 'In ICU'))

In [ ]:
df_final['Outcome'].unique()

In [ ]:
df_final['Outcome'].value_counts()

In [ ]:
df_final.to_excel('Outcome_Checking.xlsx')

In [ ]:
OutcomesMap = {'Discharged':0, 'Death':1, 'Inpatient':0, 'In ICU':1, '':0}

In [ ]:
df_final['Outcome'] = df_final['Outcome'].map(OutcomesMap)

In [ ]:
df_final['Outcome'].value_counts()

In [ ]:
df_final.Ethnicity.unique()

In [ ]:
Ethnicity_Map = {'White British':'0', 'Other White':'0', 'Unknown':'1', 'Other Black':'1',
       'White and Black African':'1', 'White and Black Caribbean':'1',
       'White Irish':'0', 'Indian':'1', 'Other':'1', 'Other Asian':'1', 'Bangladeshi':'1',
       'Black African':'1', 'Pakistani':'1', 'Chinese':'1'}

In [ ]:
df_final.Ethnicity = df_final.Ethnicity.map(Ethnicity_Map)

In [ ]:
df_final.Ethnicity.value_counts()

In [ ]:
df_final['Lymphopenia'] = df_final['LYM'] < 1

In [ ]:
df_final.Lymphopenia.dtypes

In [ ]:
df_final['Lymphopenia'].value_counts()

In [ ]:
df_final['Lymphopenia'] = df_final['Lymphopenia'].astype(int)

In [ ]:
df_final['Lymphopenia'].value_counts()

In [ ]:
df_final.Sex.unique()

In [ ]:
df_final.Sex = df_final.Sex.apply(lambda x: 1 if x == 'Male' else 0)

In [ ]:
df_final.Sex.unique()

In [ ]:
df_final.SpecificCOVID19Treatment = df_final.SpecificCOVID19Treatment.apply(lambda x: 1 if x == 'Yes' else 0)
df_final.ChronicRespiratoryDisease = df_final.ChronicRespiratoryDisease.apply(lambda x: 1 if x == 'Yes' else 0)
df_final.Asthma = df_final.Asthma.apply(lambda x: 1 if x == 'Yes' else 0)
df_final.ChronicHeartDisease = df_final.ChronicHeartDisease.apply(lambda x: 1 if x == 'Yes' else 0)
df_final.Hypertension = df_final.Hypertension.apply(lambda x: 1 if x == 'Yes' else 0)
df_final.ChronicRenalDisease = df_final.ChronicRenalDisease.apply(lambda x: 1 if x == 'Yes' else 0)
df_final.ChronicLiverDisease = df_final.ChronicLiverDisease.apply(lambda x: 1 if x == 'Yes' else 0)
df_final.ChronicNeurologicalDisease = df_final.ChronicNeurologicalDisease.apply(lambda x: 1 if x == 'Yes' else 0)
df_final.Diabetes = df_final.Diabetes.apply(lambda x: 1 if x == 'Yes' else 0)
df_final.ImmunosuppressionTreatmentRelated = df_final.ImmunosuppressionTreatmentRelated.apply(lambda x: 1 if x == 'Yes' else 0)
df_final.ImmunosuppressionDiseaseRelated = df_final.ImmunosuppressionDiseaseRelated.apply(lambda x: 1 if x == 'Yes' else 0)
df_final.ObesityClinicallyApparent = df_final.ObesityClinicallyApparent.apply(lambda x: 1 if x == 'Yes' else 0)
df_final.UnderLyingConditionsOther = df_final.UnderLyingConditionsOther.apply(lambda x: 1 if x == 'Yes' else 0)
df_final.ComplicationsOther = df_final.ComplicationsOther.apply(lambda x: 1 if x == 'Yes' else 0)

In [ ]:
df_final.UnderLyingConditionsOther.unique()

In [ ]:
df_final.info()

In [ ]:
df_final.head()

In [ ]:
df_final['Today'] = datetime.date.today()
df_final['Today'] = pd.to_datetime(df_final['Today'])

In [ ]:
df_final['Age_Days'] = df_final['Today'] - df_final['DOB']

In [ ]:
df_final['Age_Days'] = df_final['Age_Days'].dt.days

In [ ]:
df_final['Age'] = round(df_final['Age_Days']/364)

In [ ]:
df_final['AKIA'] = df_final['AKIA'].fillna(0)

In [ ]:
df_final['AKI'] = df_final['AKIA'].apply(lambda x: 0 if x == 0 else 1)

In [ ]:
lambda x: x*10 if x<2 else (x**2 if x<4 else x+10)

In [ ]:
df_final['NEWS2>5.10'] = df_final['Total Obs Score'].apply(lambda x: 1 if x >5.1 else (0 if x<=5.1 else np.nan) )
df_final['CRP > 173.6'] = df_final['CRP'].apply(lambda x: 1 if x >=173.6 else (0 if x<173.6 else np.nan) )
df_final['ALB2<31.1'] = df_final['ALB2'].apply(lambda x: 1 if x <=31.1 else (0 if x>31.1 else np.nan) )
df_final['GFR3<31.6'] = df_final['GFR3'].apply(lambda x: 1 if x <=31.6 else (0 if x>31.6 else np.nan) )
df_final['NEUT>8.77'] = df_final['NEUT'].apply(lambda x: 1 if x >8.77 else (0 if x<=8.77 else np.nan) )

In [ ]:
df_final.to_excel(Output_Dir + 'NEWS2_FullDatabase_LAB_VERSION.xlsx', index = False)

In [ ]:
df_final.info()

In [ ]:
df_final['ChronicHeartDisease']

In [ ]:
df_final['ChronicHeartDisease'] = (df_final['IfChronicHeartDiseaseSpecifyCondition'].str.contains('a'))

In [ ]:
df_final['ChronicHeartDisease'] = (df_final['ChronicHeartDisease'].fillna(False))

In [ ]:
df_final['ChronicHeartDisease'] = df_final['ChronicHeartDisease'].astype(int)

In [ ]:
import math
df_final['CRP_squareroot'] = df_final['CRP'].apply(lambda x: math.sqrt(x))

In [ ]:
df_baseline = df_final[['index_date', 'Age', 'Ethnicity','Sex',
        'ChronicRespiratoryDisease',
        'Asthma',
        'ChronicHeartDisease',
        'Hypertension','ChronicRenalDisease', 'ChronicLiverDisease',
       'ChronicNeurologicalDisease','Diabetes',
       'ImmunosuppressionTreatmentRelated',
       'ImmunosuppressionDiseaseRelated',
       'ObesityClinicallyApparent',
       'Date & Time of Receipt', '*Respiration rate', '25VD', 'AKIA',
       'ALB2', 'AST', 'CR2', 'CRP','CRP_squareroot', 'DD2', 'FER2', 'FIBD', 'GFR3', 'HST1',
       'Hb', 'LYM', 'NEUT', 'PLT', 'Peripheral oxygen saturation', 'TRIG',
       'Total Obs Score', 'WBC',
       'AKI', 'NEWS2>5.10', 'CRP > 173.6','Lymphopenia',
       'ALB2<31.1', 'GFR3<31.6', 'NEUT>8.77','Final_Outcome_Time',  'Outcome']]

In [ ]:
df_baseline['ChronicRespiratoryDisease']=df_baseline['ChronicRespiratoryDisease'].apply(lambda x: 1 if x == 'Yes' else 0)
df_baseline['Asthma']=df_baseline['Asthma'].apply(lambda x: 1 if x == 'Yes' else 0)
df_baseline['ChronicHeartDisease']=df_baseline['ChronicHeartDisease'].apply(lambda x: 1 if x == 'Yes' else 0)
df_baseline['Hypertension']=df_baseline['Hypertension'].apply(lambda x: 1 if x == 'Yes' else 0)
df_baseline['ChronicRenalDisease'] =df_baseline['ChronicRenalDisease'].apply(lambda x: 1 if x == 'Yes' else 0)
df_baseline['ChronicLiverDisease']=df_baseline['ChronicLiverDisease'].apply(lambda x: 1 if x == 'Yes' else 0)
df_baseline['ChronicNeurologicalDisease']=df_baseline['ChronicNeurologicalDisease'].apply(lambda x: 1 if x == 'Yes' else 0)
df_baseline['Diabetes']= df_baseline['Diabetes'].apply(lambda x: 1 if x == 'Yes' else 0)
df_baseline['ImmunosuppressionTreatmentRelated']= df_baseline['ImmunosuppressionTreatmentRelated'].apply(lambda x: 1 if x == 'Yes' else 0)
df_baseline['ImmunosuppressionDiseaseRelated']=df_baseline['ImmunosuppressionDiseaseRelated'].apply(lambda x: 1 if x == 'Yes' else 0)
df_baseline['ObesityClinicallyApparent']=df_baseline['ObesityClinicallyApparent'].apply(lambda x: 1 if x == 'Yes' else 0)
       

In [ ]:
df_baseline.to_csv(Output_Dir + 'BaselineNew.csv', index = False)